## 가온 차트 제목, 가수, 가온지수, 가사 크롤링 코드

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url = "http://gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL&targetTime=23&hitYear=2019&termGbn=week#"
req = requests.get(url)

html = req.text
soup = BeautifulSoup(html, 'html.parser')

In [13]:
# 제목 크롤링

titles = []

subject = soup.find_all("td", {"class": "subject"})

for s in subject:
    p = s.find_all('p')
    titles.append(p[0].text.strip())

titles[:10]

['사랑에 연습이 있었다면 (Prod. 2soo)',
 '누구 없소 (NO ONE) (Feat. B.I of iKON)',
 '2002',
 '너에게 못했던 내 마지막 말은',
 'bad guy',
 '작은 것들을 위한 시 (Boy With Luv) (Feat. Hals...',
 '주저하는 연인들을 위해',
 'FANCY',
 'Goodbye',
 '노래방에서']

In [14]:
# 가수 크롤링

singers = []

subject = soup.find_all("td", {"class": "subject"})

for s in subject:
    p = s.find_all('p')
    singers.append(p[1].text.split('|')[0])

singers[:10]

['임재현',
 '이하이',
 'Anne-Marie',
 '다비치',
 'Billie Eilish',
 '방탄소년단',
 '잔나비',
 'TWICE',
 '박효신',
 '장범준']

In [18]:
# 가온지수 크롤링

counts = []

count = soup.find_all("td", {"class": "count"})

for c in count:
    p = c.find_all('p')
    counts.append(p[0].text)

counts[:10]

['44,009,459',
 '43,074,538',
 '41,877,109',
 '38,165,624',
 '33,544,212',
 '32,741,579',
 '28,586,209',
 '25,750,142',
 '23,823,214',
 '23,558,147']

In [19]:
# 가사 클릭 링크 크롤링

links = []

play = soup.find_all("td", {"class": "play"})

for p in play:
    link_tag = p.find_all('a')[0]
    if 'href' in link_tag.attrs:
        links.append(link_tag.attrs['href'])
        
links[:10]

['javascript:go_returnUrl("3715","1655389");',
 'javascript:go_returnUrl("3715","1777099");',
 'javascript:go_returnUrl("3715","1512809");',
 'javascript:go_returnUrl("3715","1771237");',
 'javascript:go_returnUrl("3715","1748016");',
 'javascript:go_returnUrl("3715","1754120");',
 'javascript:go_returnUrl("3715","1740697");',
 'javascript:go_returnUrl("3715","1760413");',
 'javascript:go_returnUrl("3715","1766839");',
 'javascript:go_returnUrl("3715","1744812");']

In [ ]:
import time
from tqdm.notebook import tqdm
from selenium import webdriver

path = "C:/Users/user/chromedriver_win32/chromedriver"
driver = webdriver.Chrome(path)

url = "http://gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL&targetTime=23&hitYear=2019&termGbn=week#"
driver.get(url)

lyrics = []

for i in tqdm(range(len(titles))):
    
    try:
        driver.execute_script(links[i])
        driver.switch_to_window(driver.window_handles[-1])
        time.sleep(1)
        
        # 노래 제목이 같고, 링크 버튼 클래스를 포함하는 a 태그 호출
        click = driver.find_element_by_xpath("//a[contains(@title, '{}')][contains(@class, 'btn')]".format(titles[i].split('.')[0]))
        click.click()

        lyrics.append(driver.find_element_by_class_name("lyric").text)
        
        driver.close()
        time.sleep(1)
        
        driver.switch_to_window(driver.window_handles[0])
       
    except:
        lyrics.append('x')
        
        driver.close()
        driver.switch_to_window(driver.window_handles[0]) 
        

In [70]:
# 데이터 합치기

chart = pd.DataFrame( data = {'title': titles, 'singer': singers, 'count': counts, 'lyrics' : lyrics}) ; chart                    

,title,singer,lyrics
0,사랑에 연습이 있었다면 (Prod. 2soo),임재현,사랑에 연습이 있었다면\n우리는 달라졌을까\n내가 널 만난 시간 혹은 그 장소\n상...
1,누구 없소 (NO ONE) (Feat. B.I of iKON),이하이,누구 없소\n나를 붙잡아줄 님은 없소\n눈을 씻고 찾아봐도 보이지가 않는데\n어디 ...
2,2002,Anne-Marie,I will always remember\nThe day you kissed my ...
3,너에게 못했던 내 마지막 말은,다비치,아직 남았을까\n못 잊을 이유가 남아 있을까\n나만 왜 이렇게\n바보같이 혼자 널 ...
4,bad guy,Billie Eilish,White shirt now red\nmy bloody nose\nSleepin'\...
...,...,...,...
195,마지막처럼,BLACKPINK,너 뭔데 자꾸 생각나\n자존심 상해 애가 타\n얼굴이 뜨겁고 가슴은 계속 뛰어\n내...
196,Bellyache,Billie Eilish,x
197,행복,오반 (OVAN),난 포기가 쉽죠 적응이 빨라서\n착한 척 하는가 싶고\n필요도 없는 생각이 넘 많아...
198,나들이 갈까,볼빨간사춘기,어디야 너 지금 뭐하고 있는 거야\n오늘 이 시간쯤엔 연락 올 줄 알았어\n점심 먹...


In [71]:
chart[chart['lyrics'] == "x"]

,title,singer,lyrics
63,봄 (Feat. 산다라박),박봄,x
89,FRIENDS,"Marshmello, Anne-Marie",x
106,Intro _ Persona,방탄소년단,x
107,Idontwannabeyouanymore,Billie Eilish,x
108,Youth,Troye Sivan,x
148,Sunflower (Spider-Man _ Into The Spide...,Post Malone & Swae Lee,x
156,"미운 날 (The day, I hate myself)",닐로 (Nilo),x
166,IndiGO,"저스디스 (JUSTHIS), Kid Milli, NO:EL, 영비 (Young B)",x
196,Bellyache,Billie Eilish,x


In [72]:
# 엑셀 파일로 저장

chart.to_excel("Gaon_chart_201906.xlsx") 